DESSERT MEDICAUX: utilisant les adresses des praticiens pour trouver les coordonnées gps

In [ ]:
from lib2to3.pgen2 import driver
import requests
from bs4 import BeautifulSoup
from matplotlib.pyplot import text
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from msedge.selenium_tools import EdgeOptions, Edge
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import folium
from folium.plugins import MarkerCluster
import pandas as pd
from lxml import etree
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
from branca.element import Figure

In [2]:
start = time.time()
dep = 44
url = f"http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/chirurgiens-dentistes/{dep}-loire-atlantique"
base_url = "http://annuairesante.ameli.fr"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
a_href = soup.findAll('a')

list_url_ville = []
list_ville = []
a_href = a_href[3:-6]
for link in a_href:
    if link.has_attr('href'):
        list_url_ville.append(link['href'])
        list_ville.append(link.text) 
  

In [4]:
def get_praticien_ville(ville, url):
    response_name = requests.get(base_url + url)
    soup_name = BeautifulSoup(response_name.text, 'html.parser')
    name = soup_name.findAll('a')
    adress = soup_name.findAll(class_="item left adresse")
    
    # contient le nom et le prenom de tous les praticiens de la ville
    name_clean = [] 
    for n in name:
        if n.strong:
            name_clean.append(n)
    
    liste_adresse=[]
    for line_break in adress:
                words = line_break.getText(
                    separator='|br|', strip=True).split('|br|')
                words = words[1:]
                str1 = ','.join(words)
                liste_adresse.append(str1)
            
    # liste des informations des praticiens de la ville: nom, adresse et ville
    list_info_praticiens = [] 
    for j in range(len(name_clean)):
        
        info_praticiens = {}        
        info_praticiens['nom'] = name_clean[j].text
        info_praticiens['adress'] = liste_adresse[j]
        info_praticiens['ville'] = ville
        list_info_praticiens.append(info_praticiens)
        
    return list_info_praticiens

In [5]:
all_praticiens = []
for i in range(len(list_ville)):
    praticiens = get_praticien_ville(list_ville[i], list_url_ville[i])
    all_praticiens.extend(praticiens)
df = pd.DataFrame(all_praticiens)
df

,nom,adress,ville
0,GUERIN ALEXIS,"11 RUE MARIE CURIE,44140 AIGREFEUILLE SUR MAINE",Aigrefeuille Sur Maine (44140)
1,BORDRON ANTOINE,"11 RUE MARIE CURIE,44140 AIGREFEUILLE SUR MAINE",Aigrefeuille Sur Maine (44140)
2,ABIZMIL YOANN,"11 RUE MARIE CURIE,44140 AIGREFEUILLE SUR MAINE",Aigrefeuille Sur Maine (44140)
3,DIDENKO MARIYA,"334 RUE DU GENERAL LECLERC,44150 ANCENIS",Ancenis (44150)
4,MORIN CLARA,"140 PLACE MAURICE GELINEAU,44150 ANCENIS",Ancenis (44150)
...,...,...,...
660,BUGEL EMMA,"8 BOULEVARD CHARLES DE GAULLE,44360 VIGNEUX DE...",Vigneux De Bretagne (44360)
661,GUERRY OLIVIA,"8 BOULEVARD CHARLES DE GAULLE,44360 VIGNEUX DE...",Vigneux De Bretagne (44360)
662,FOURNEL ALEXANDRA,"37B RUE DU PETIT PRINCE,44360 VIGNEUX DE BRETAGNE",Vigneux De Bretagne (44360)
663,GUIBOUIN CECILIA,"1 IMPASSE DU HARAS,44640 VUE",Vue (44640)


In [ ]:
res=df.adress
res=list(set(res))

def get_dpe_from_url(url):

        req = requests.get(url)
        wb = req.json()
        df = pd.json_normalize(wb["features"])
        return df
    
m2 = folium.Map(location=[46.227638,2.213749], tiles="OpenStreetMap", zoom_start=5)
marker_cluster = MarkerCluster().add_to(m2)
for i in range(len(res)):
    url_api = f"https://api-adresse.data.gouv.fr/search/?q={res[i]}"
    dpe = get_dpe_from_url(url_api)
    coordinates=dpe['geometry.coordinates']
    lat_long=np.array(coordinates)
    tooltip = res[i]
    folium.Marker(location=[lat_long[0][1],lat_long[0][0]], tooltip=tooltip).add_to(marker_cluster)

#m2